In [1]:
import tensorflow as tf
from tensorflow import keras
import io
import requests
from PIL import Image
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np


2022-01-29 00:27:15.805504: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-29 00:27:15.805531: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

2022-01-29 00:27:40.588177: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-29 00:27:40.588226: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pranav-Nitro-AN515-55): /proc/driver/nvidia/version does not exist
2022-01-29 00:27:40.588855: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
TRAINING_DIR = "./train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [4]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [5]:
history = model.fit(train_generator,epochs=10,validation_data=validation_generator,callbacks=[checkpoint])

Epoch 1/10
132/132 [==============================] - ETA: 0s - loss: 0.5800 - acc: 0.6971

2022-01-29 00:29:20.559360: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model2-001.model/assets
132/132 [==============================] - 88s 654ms/step - loss: 0.5800 - acc: 0.6971 - val_loss: 0.2718 - val_acc: 0.9433
Epoch 2/10
132/132 [==============================] - 81s 613ms/step - loss: 0.4312 - acc: 0.8318 - val_loss: 0.1919 - val_acc: 0.9227
Epoch 3/10
132/132 [==============================] - 83s 629ms/step - loss: 0.3051 - acc: 0.8836 - val_loss: 0.1960 - val_acc: 0.9381
Epoch 4/10
132/132 [==============================] - 81s 610ms/step - loss: 0.2877 - acc: 0.8919 - val_loss: 0.1658 - val_acc: 0.9742
Epoch 5/10
132/132 [==============================] - 88s 667ms/step - loss: 0.2589 - acc: 0.8965 - val_loss: 0.0978 - val_acc: 0.9588
Epoch 6/10
132/132 [==============================] - 83s 631ms/step - loss: 0.2333 - acc: 0.9117 - val_loss: 0.1053 - val_acc: 0.9536
Epoch 7/10
132/132 [==============================] - 68s 517ms/step - loss: 0.1998 - acc: 0.9247 - val_loss: 0.0643 - val_acc: 0.9794
Epoch 8

In [6]:
model.save('/home/pranav/Desktop/Untitled Folder/model')

INFO:tensorflow:Assets written to: /home/pranav/Desktop/Untitled Folder/model/assets


In [1]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("model")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()

2022-01-29 00:52:52.087358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pranav/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-01-29 00:52:52.087379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-01-29 00:52:53.606805: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-29 00:52:53.606861: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pranav-Nitro-AN515-55): /proc/driver/nvidia/version does not exist
2022-01-29 00:52:53.607261: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Librar

KeyboardInterrupt: 